Importing Data and packages

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import parameters as params

base_der = pd.read_csv('data/case1/base-case-DER.csv', skiprows=1, usecols=[0, 1, 2, 3], header=None)
base_load = pd.read_csv('data/case1/base-case-load.csv', skiprows=1, usecols=[0, 1, 2, 3, 4], header=None)
base_line = pd.read_csv('data/case1/base-case-line.csv', skiprows=1, usecols=[0, 1, 2, 3, 4, 5], header=None)

DERs = set()
CLs = set()

for u in base_der[0]:
    DERs.add(u)

for load, precedance in zip(base_load[0], base_load[1]):
    if precedance == 'Priority':
        CLs.add(load)

Adding edges and weights to graph

In [ ]:
G = nx.Graph()
wt = {}
for u, v, w in zip(base_line[0], base_line[1], base_line[3]):
    G.add_edge(u, v)
    wt[(u, v)] = w
    wt[(v, u)] = w

Finding all simple paths from each critical load to each source

In [ ]:
path_list = []
for cl in CLs:
    for path in nx.all_simple_paths(G, source=cl, target=DERs):
        loads, sources = 0, 0
        for node in path:
            if node in CLs:
                loads += 1
            if node in DERs:
                sources += 1
        if loads > 1 or sources > 1:
            continue
        path_list.append(list(nx.utils.pairwise(path)))
print(path_list)


All possible path combination calculation

In [ ]:
from collections import Counter


possible_networks = set()
comb = Counter()
def find_networks(comb: Counter, start: int):
    if start == len(path_list):
        return
    possible_networks.add(tuple(comb.keys()))
    for i in range(start, len(path_list)):
        path = path_list[i]
        comb.update(path)
        find_networks(comb, i+1)
        comb.subtract(path)
        comb = Counter([c for c in comb.elements() if comb[c] > 0])
        find_networks(comb, i+1)

find_networks(comb, 0)
FNs = []
for i, fn in enumerate(possible_networks):
    print(f"FN{i}: ", list(fn))
    FNs.append(list(fn))



In [ ]:
pos = nx.spring_layout(G)
nx.draw_networkx(G, pos, font_color = 'w')
nx.draw_networkx_nodes(G, pos, list(DERs), node_color='g')
nx.draw_networkx_nodes(G, pos, list(CLs), node_color='r')
nx.draw_networkx_edges(G, pos, FNs[35], width=4)
plt.show()

### Calculating parameters to measure resiliency of possible network

Calculation of Parameter matrix

In [ ]:
rsl = params.ratio_source_load(FNs, CLs, DERs)
ops = params.switch_ops(FNs, wt)
cen = params.agg_centrality(FNs)

df = pd.DataFrame(np.array([rsl, ops, cen]), columns=[f"FN{i}" for i in range(1, len(rsl)+1)])
df.index = ['rsl', 'ops', 'cen']
df = df.T
print(df)

Calculation of weighted mean and resiliency

In [ ]:
df['wtd_mean'] = (df['rsl'] * 4 + df['ops'] * 3 + df['cen'] * 3)/10
df['resiliency'] = np.exp(-df['wtd_mean'])
print(df)

Finding critical lines based on network topology